In [1]:
#0. import von relevanten Bibliotheken
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression

from sklearn.tree import DecisionTreeRegressor
from sklearn import tree 

## Woche 3 - Machine Learning Wrap-Up

In dieser Woche betrachten wir die Lineare Regession und Decision Trees als Modelle, welche uns helfen sollen 
das Esemble aus Indikatoren zu dirigieren.

#### Training und Test Datensatz

#### Lineare Regression


#### Decision Trees




In [2]:
#Definition zum Simple Moving Average (SMA)
def sma(data: pd.DataFrame,intervall:int):
  #diese Funktion berechnet den SMA_intervall von dem Input data
  #der SMA wird immer von dem Column "4a. close (EUR)" berechnet
  #ACHTUNG: Stelle sicher, dass data immer diese Spalte besitzt
  spalten_name = "SMA_"+str(intervall)
  data[spalten_name] = data["4a. close (EUR)"].rolling(intervall).mean()

#Definition des Relative Strength Index (RSI)
def RSI_berechnen(data:pd.DataFrame,intervall:int):

  spalten_name = "RSI_"+str(intervall)
  data[spalten_name] = data["4a. close (EUR)"].rolling(intervall).mean()

  # Bestimme die Preisänderung zum jeweiligen Zeitpunkt t-1
  delta = data["4a. close (EUR)"].diff()

  # Get rid of the first row, which has NaN values
  delta = delta[1:]

  # Calculate the gains and losses
  up = delta.where(delta > 0, 0)
  down = -delta.where(delta < 0, 0)

  # Calculate the rolling average of the gains and losses
  #window_size = 14 #als default
  avg_gain = up.rolling(intervall).mean()
  avg_loss = down.rolling(intervall).mean()

  # Calculate the relative strength
  rs = avg_gain / avg_loss

  # Calculate the RSI
  data[spalten_name] = 100 - (100 / (1 + rs))


In [ ]:
#1. laden von dem Datensatz
daten = pd.read_csv("daten_07_05_2020_bis_31_01_2023.csv",index_col="date")
daten = daten.sort_index(axis=0)
df = daten[["4a. close (EUR)"]].copy()
df.head()

In [84]:
#Indikatoren bestimmen
for i in range(2,21):
    RSI_berechnen(data=df,intervall=i)
    sma(data=df,intervall=i)
#pct Change bestimmen
df["pct"] = df["4a. close (EUR)"].pct_change().shift(-1)

df = df.dropna()

In [ ]:
df.head(2)

In [91]:
#Aufteilen des Datensatzes in Train (bekannter Teil) und Test (unbekannt)
train_size = int(len(df) * 0.8)
df_train = df.iloc[:train_size]
df_test = df.iloc[train_size:]

In [ ]:
#for loop Daten review
# Statistisch Sinnvoll den RSI zu verwenden?

for k in range(4,21):
    print("\nNEUER RSI" ,k)
    my_rsi="RSI_"+str(k)

    grenze = 25.0
    grenze_ob = 100 - grenze
    print(f"{my_rsi} {grenze}-Quantil:\n",df_train["pct"][df_train[my_rsi]<grenze].describe())
    print("\npositiv in nächster Periode:" , (df_train["pct"].where(df_train[my_rsi]<grenze)>=0.0).value_counts()[1])
    print("negativ in nächster Periode:" , (df_train["pct"].where(df_train[my_rsi]<grenze)<0.0).value_counts()[1])
    print("Quotient: ",(df_train["pct"].where(df_train[my_rsi]<grenze)>=0.0).value_counts()[1]/(df_train["pct"].where(df_train[my_rsi]<grenze)<0.0).value_counts()[1])

    print("-- -- -- -- -- -- --")

    print(f"\n{my_rsi} {grenze_ob}-Quantil:\n",df_train["pct"][df_train[my_rsi]>grenze_ob].describe())
    print("\npositiv in nächster Periode:" , (df_train["pct"].where(df_train[my_rsi]>grenze_ob)>=0.0).value_counts()[1])
    print("negativ in nächster Periode:" , (df_train["pct"].where(df_train[my_rsi]>grenze_ob)<0.0).value_counts()[1])
    print("Quotient: ",(df_train["pct"].where(df_train[my_rsi]>grenze_ob)<0.0).value_counts()[1]/(df_train["pct"].where(df_train[my_rsi]>grenze_ob)>=0.0).value_counts()[1])

    plt.figure(k)
    plt.scatter(df_train[my_rsi].where(df_train[my_rsi]<grenze),df_train["pct"].where(df_train[my_rsi]<grenze))
    plt.axhline(color="r")
    plt.axhline(df_train["pct"][df_train[my_rsi]<grenze].median(), xmax=grenze/100.00,color="black")
    plt.axhline(df_train["pct"][df_train[my_rsi]<grenze].mean(), xmax=grenze/200.00,color="cyan")

    plt.scatter(df_train[my_rsi].where(df_train[my_rsi]>grenze_ob),df_train["pct"].where(df_train[my_rsi]>grenze_ob))
    plt.axhline(color="r")
    plt.axhline(df_train["pct"][df_train[my_rsi]>grenze_ob].median(), xmin=grenze_ob/100.00,color="black")
    plt.axhline(df_train["pct"][df_train[my_rsi]>grenze_ob].mean(), xmin=grenze_ob/85.00,color="cyan")

    plt.show()

In [ ]:
# Train Daten RSI_17
my_rsi="RSI_"+str(17)

grenze = 25.0
grenze_ob = 100 - grenze
print(f"{my_rsi} {grenze}-Quantil:\n",df_train["pct"][df_train[my_rsi]<grenze].describe())
print("\npositiv in nächster Periode:" , (df_train["pct"].where(df_train[my_rsi]<grenze)>=0.0).value_counts()[1])
print("negativ in nächster Periode:" , (df_train["pct"].where(df_train[my_rsi]<grenze)<0.0).value_counts()[1])
print("Quotient: ",(df_train["pct"].where(df_train[my_rsi]<grenze)>=0.0).value_counts()[1]/(df_train["pct"].where(df_train[my_rsi]<grenze)<0.0).value_counts()[1])

print("-- -- -- -- -- -- --")

print(f"\n{my_rsi} {grenze_ob}-Quantil:\n",df_train["pct"][df_train[my_rsi]>grenze_ob].describe())
print("\npositiv in nächster Periode:" , (df_train["pct"].where(df_train[my_rsi]>grenze_ob)>=0.0).value_counts()[1])
print("negativ in nächster Periode:" , (df_train["pct"].where(df_train[my_rsi]>grenze_ob)<0.0).value_counts()[1])
print("Quotient: ",(df_train["pct"].where(df_train[my_rsi]>grenze_ob)<0.0).value_counts()[1]/(df_train["pct"].where(df_train[my_rsi]>grenze_ob)>=0.0).value_counts()[1])

plt.scatter(df_train[my_rsi],df_train["pct"])
plt.scatter(df_train[my_rsi].where(df_train[my_rsi]<grenze),df_train["pct"].where(df_train[my_rsi]<grenze))
plt.scatter(df_train[my_rsi].where(df_train[my_rsi]>grenze_ob),df_train["pct"].where(df_train[my_rsi]>grenze_ob))

plt.axhline(color="r")
plt.axvline(grenze, ymin=0.3, ymax=0.8,color="y")
plt.axvline(grenze_ob,ymin=0.3, ymax=0.8,color="y")

plt.axhline(df_train["pct"][df_train[my_rsi]<grenze].median(), xmax=grenze/100.00,color="black")
plt.axhline(df_train["pct"][df_train[my_rsi]<grenze].mean(), xmax=grenze/200.00,color="cyan")
plt.axhline(df_train["pct"][df_train[my_rsi]>grenze_ob].median(), xmin=grenze_ob/100.00,color="black")
plt.axhline(df_train["pct"][df_train[my_rsi]>grenze_ob].mean(), xmin=grenze_ob/85.00,color="cyan")


In [ ]:
#Test Daten RSI_17
my_rsi="RSI_"+str(17)

grenze = 25.0
grenze_ob = 100 - grenze
print(f"{my_rsi} {grenze}-Quantil:\n",df_test["pct"][df_test[my_rsi]<grenze].describe())
print("\npositiv in nächster Periode:" , (df_test["pct"].where(df_test[my_rsi]<grenze)>=0.0).value_counts()[1])
print("negativ in nächster Periode:" , (df_test["pct"].where(df_test[my_rsi]<grenze)<0.0).value_counts()[1])
print("Quotient: ",(df_test["pct"].where(df_test[my_rsi]<grenze)>=0.0).value_counts()[1]/(df_test["pct"].where(df_test[my_rsi]<grenze)<0.0).value_counts()[1])

print("-- -- -- -- -- -- --")

print(f"\n{my_rsi} {grenze_ob}-Quantil:\n",df_test["pct"][df_test[my_rsi]>grenze_ob].describe())
print("\npositiv in nächster Periode:" , (df_test["pct"].where(df_test[my_rsi]>grenze_ob)>=0.0).value_counts()[1])
print("negativ in nächster Periode:" , (df_test["pct"].where(df_test[my_rsi]>grenze_ob)<0.0).value_counts()[1])
print("Quotient: ",(df_test["pct"].where(df_test[my_rsi]>grenze_ob)<0.0).value_counts()[1]/(df_test["pct"].where(df_test[my_rsi]>grenze_ob)>=0.0).value_counts()[1])

plt.scatter(df_test[my_rsi],df_test["pct"])
plt.scatter(df_test[my_rsi].where(df_test[my_rsi]<grenze),df_test["pct"].where(df_test[my_rsi]<grenze))
plt.scatter(df_test[my_rsi].where(df_test[my_rsi]>grenze_ob),df_test["pct"].where(df_test[my_rsi]>grenze_ob))

plt.axhline(color="r")
plt.axvline(grenze, ymin=0.3, ymax=0.8,color="y")
plt.axvline(grenze_ob,ymin=0.3, ymax=0.8,color="y")

plt.axhline(df_test["pct"][df_test[my_rsi]<grenze].median(), xmax=grenze/100.00,color="black")
plt.axhline(df_test["pct"][df_test[my_rsi]<grenze].mean(), xmax=grenze/200.00,color="cyan")
plt.axhline(df_test["pct"][df_test[my_rsi]>grenze_ob].median(), xmin=grenze_ob/100.00,color="black")
plt.axhline(df_test["pct"][df_test[my_rsi]>grenze_ob].mean(), xmin=grenze_ob/85.00,color="cyan")


In [ ]:
#Gesamt

my_rsi="RSI_"+str(17)

grenze = 25.0
grenze_ob = 100 - grenze
print(f"{my_rsi} {grenze}-Quantil:\n",df["pct"][df[my_rsi]<grenze].describe())
print("\npositiv in nächster Periode:" , (df["pct"].where(df[my_rsi]<grenze)>=0.0).value_counts()[1])
print("negativ in nächster Periode:" , (df["pct"].where(df[my_rsi]<grenze)<0.0).value_counts()[1])
print("Quotient: ",(df["pct"].where(df[my_rsi]<grenze)>=0.0).value_counts()[1]/(df["pct"].where(df[my_rsi]<grenze)<0.0).value_counts()[1])

print("-- -- -- -- -- -- --")

print(f"\n{my_rsi} {grenze_ob}-Quantil:\n",df["pct"][df[my_rsi]>grenze_ob].describe())
print("\npositiv in nächster Periode:" , (df["pct"].where(df[my_rsi]>grenze_ob)>=0.0).value_counts()[1])
print("negativ in nächster Periode:" , (df["pct"].where(df[my_rsi]>grenze_ob)<0.0).value_counts()[1])
print("Quotient: ",(df["pct"].where(df[my_rsi]>grenze_ob)<0.0).value_counts()[1]/(df["pct"].where(df[my_rsi]>grenze_ob)>=0.0).value_counts()[1])

plt.scatter(df[my_rsi],df["pct"])
plt.scatter(df[my_rsi].where(df[my_rsi]<grenze),df["pct"].where(df[my_rsi]<grenze))
plt.scatter(df[my_rsi].where(df[my_rsi]>grenze_ob),df["pct"].where(df[my_rsi]>grenze_ob))

plt.axhline(color="r")
plt.axvline(grenze, ymin=0.3, ymax=0.8,color="y")
plt.axvline(grenze_ob,ymin=0.3, ymax=0.8,color="y")

plt.axhline(df["pct"][df[my_rsi]<grenze].median(), xmax=grenze/100.00,color="black")
plt.axhline(df["pct"][df[my_rsi]<grenze].mean(), xmax=grenze/200.00,color="cyan")
plt.axhline(df["pct"][df[my_rsi]>grenze_ob].median(), xmin=grenze_ob/100.00,color="black")
plt.axhline(df["pct"][df[my_rsi]>grenze_ob].mean(), xmin=grenze_ob/85.00,color="cyan")


### Regression

Die lineare Regression ist eine statistische Methode, welche die Beziehung zwischen Variablen analisiert.
Mathematische Formel:

$ y = \beta_0 + \beta_1 * x_1 + ... + \beta_n * x_n + \epsilon$

wobei $\beta_0$ der sogenannte "Intercept", $x=(x_1,...,x_n)$ der Input Vektor, $(\beta_1,..., \beta_n)$ die Koeffizienten und $\epsilon$ den Fehlerterm beschreiben.

Bildlich gesprochen, versucht man mit der Regression eine "Gerade" zu finden, welche die Distanz zu den Datenpunkten minimiert.

In [ ]:
#Linear Regession

#Training und Test Size aufteilen
train_size = int(len(df) * 0.8)
train_data = df.iloc[:train_size]
test_data = df.iloc[train_size:]

model = LinearRegression()
model.fit(train_data[["RSI_17"]], train_data["pct"])

y = b_0 + b_1 * x

y_pred = 0.00567 + (-0.0000743) * x

In [ ]:
nst = model.intercept_/((-1)*model.coef_)
nst

In [ ]:
#Was unsere Regression macht:
X_train = train_data["RSI_17"].values[:,np.newaxis]
y_train = train_data["pct"].values

plt.scatter(X_train,y_train,color="g")
plt.plot(X_train,model.predict(X_train),color="black")
plt.axhline(color="r")

nst = model.intercept_/((-1)*model.coef_)
nst

In [12]:
#predict
y1 = model.predict(df[["RSI_17"]].iloc[train_size:]) 

In [ ]:
#PF bestimmen
PF = pd.DataFrame(df["pct"].iloc[train_size:])
PF["pred"] = y1

PF["PF_17"] = (PF["pred"]>0.0) * PF["pct"]
PF["myPF_17"] = (PF["PF_17"] +1).cumprod()

PF["PF_BM"] =  PF["pct"]
PF["myPF_BM"] = (PF["PF_BM"] +1).cumprod()

PF.tail()

In [ ]:
#Plotte dein Ergebnis
PF[["myPF_BM","myPF_17"]].plot(figsize=(16,6))

## Decision Tree

Decision trees sind eine powervolle ML Methode, die sowohl für eine Klassifikation als auch eine Regression verwendet werden kann.

Ein Vorteil des Decision Trees gegenüber der Regression ist, dass er sogar mit fehlenden (NAN) Werten arbeiten kann. Jedoch neigt er zu einem Overfitting, wenn die "depth-size" (Tiefe) des Baums zu hoch gewählt wird.


Wir gehen im folgenden auf einen Decision Tree ein:





In [ ]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt

my_rsi = "RSI_" + str(17)

# Aufteilen der Daten in Trainings- und Testdaten
train_size = int(len(df) * 0.8)
train_data = df.iloc[:train_size]
test_data = df.iloc[train_size:]

X_train = train_data[[my_rsi,"SMA_17","SMA_10","SMA_15","RSI_10"]].to_numpy()      #X_train = train_data[[my_rsi,"SMA_17"]].to_numpy()
Y_train = train_data["pct"].to_numpy()

X_test = test_data[[my_rsi,"SMA_17","SMA_10","SMA_15","RSI_10"]].to_numpy()         #X_test = test_data[[my_rsi,"SMA_17"]].to_numpy()
Y_test = test_data["pct"].to_numpy()


# Fit regression model
regr_1 = DecisionTreeRegressor(max_depth=5,random_state=1234)
regr_2 = DecisionTreeRegressor(max_depth=30,random_state=1234)    
regr_1.fit(X_train, Y_train)
regr_2.fit(X_train, Y_train)

In [ ]:
#plotten des Baums
plt.figure(figsize=(16,10))
tree.plot_tree(regr_1,fontsize=8)
plt.show()

In [ ]:
# Die "Güte" der Modelle nach R^2
print("Model 1:")
print(regr_1.score(X_train,y_train))
print(regr_1.score(X=X_test,y=Y_test))

print("\nModel 2:")
print(regr_2.score(X_train,y_train))
print(regr_2.score(X=X_test,y=Y_test))

In [ ]:
regr_1.feature_importances_

In [ ]:
regr_2.feature_importances_

In [77]:
#predict
y_1 = regr_1.predict(X_test) 
y_2 = regr_2.predict(X_test)

In [ ]:
PF = pd.DataFrame(df["pct"].iloc[train_size:])
PF["pred"] = y_1
PF["pred2"] = y_2

PF["PF_17"] = (PF["pred"]>0.0) * PF["pct"]
PF["myPF_17"] = (PF["PF_17"] +1).cumprod()

PF["PF_17v2"] = (PF["pred2"]>0.0) * PF["pct"]
PF["myPF_17v2"] = (PF["PF_17v2"] +1).cumprod()

PF["PF_BM"] =  PF["pct"]
PF["myPF_BM"] = (PF["PF_BM"] +1).cumprod()

PF.tail()

In [ ]:
PF[["myPF_BM","myPF_17","myPF_17v2"]].plot(figsize=(16,6))

In [ ]:
#TODO I: - Regression
# a) Trainiere dein Modell mit dem RSI Wert 10. 
#    Beschreibe die bei welchen Werten des RSI ein Kauf empfohlen wird. 
# b) Beschreibe in eigenen Worte, was die Länge des betrachteten Intervall auf den RSI für einen
#    Einfluss hat. Wann wird im allgemeinen häufiger ein Überkaufter Wert erreicht? Bei längeren
#    oder kürzeren Intervallen?

In [ ]:
#TODO II: - Decission Tree
# a) Bei der maxDepth von 5 wird ein Feature mit der importance von 0 angegeben. Was passiert, wenn
#    du dieses aus den Input-Daten für dein Model herausnimmst? Plotte den Graph für beide Modelle 
#    (mit maxDepth= 5 und 30). Erkläre in eigenen Worten was passiert ist.
# b) Erkläre in eigenen Worten, was der maxDepth Parameter bedeutet. Ist es sinnvoll diesen immer
#    maximal zu wählen?

### Ausblick vierte Woche
- OBV Indikator
- Bot fertigstellen

- eventuelle: Backtesting Paper schreiben